# データの読み込み

In [1]:
import copy
import numpy as np
import itertools
import math
import random






#ナンプレの問題をboardに保存
def data_input(n):
    global board_all,input_data,blank_count_array
    input_data =[[]for _ in range(n)]
    data =list(input())
    blank_count_2000 =data.count("0")/2000 #すべてのナンプレにおける空欄の数の平均
    blank_count_array =[] #空欄数を保存
    
    board_all =[[[]for _ in range(9)] for _ in range(n)]
    new_line_error =0
    for number in range(n):
        row =0
        blank_count =0 #１つのナンプレの空欄数
        for i in range(81):
            if i %9==0 and i !=0:
                row +=1
            if data[i+number*81+new_line_error] =="0":
                board_all[number][row].append(" ")
                input_data[number].append("0")
            else:
                board_all[number][row].append(str(data[i+number*81+new_line_error]))
                input_data[number].append(str(data[i+number*81+new_line_error]))
            if data[i+number*81+new_line_error] =="0":
                blank_count +=1
        blank_count_array.append(blank_count)
        
        new_line_error +=1
        input_data[number] ="".join(input_data[number])
    return board_all,input_data,blank_count,blank_count_array
def ans_candidate_set(board):
    global ans_candidate
    pre =set(["1","2","3","4","5","6","7","8","9"])
    ans_candidate =[[copy.copy(pre) for _ in range(9)] for _ in range(9)]#答えの候補
    row_common_sense(ans_candidate,board)
    colum_common_sense(ans_candidate,board)
    block_common_sense(ans_candidate,board)
    for i in range(9):
        for j in range(9):
            if board[i][j] !=" ":
                ans_candidate[i][j] =set([])
    ans_candidate =np.array(list(ans_candidate)).reshape(9,9)
    return ans_candidate



# 盤面が全て埋まっているか？候補が１つでないか？行、列、ブロックの常識

In [2]:

#全て数字が埋まっているかの判定
def finish(board):
    for row in range(9):
        for colum in range(9):
            if board[row][colum]==" ":
                return False
    return True

#残り物の常識
def leftovers(ans_candidate,board):
    flag_leftovers =False
    for row in range(9):
        for colum in range(9):
            if board[row][colum]==" ":
                if len(list(ans_candidate[row][colum]))==1:
                    flag_leftovers =True
                    board[row][colum]  =ans_candidate[row][colum].pop()
                    ans_candidate[row][colum] =set([])
            else:
                 ans_candidate[row][colum] =set([])

    return flag_leftovers
            

#行の常識               
def row_common_sense(ans_candidate,board):
    flag_row_common_sense =False
    #各行の数字を取得
    for row in range(9):
        board_alredy_in =set([])
        for colum in range(9):
            if board[row][colum]!=" ":
                board_alredy_in.add(board[row][colum])

        #行の常識を使い、候補を絞り込む
        for i in range(9):
            if board[row][i] ==" ":
                if ans_candidate[row][i] - board_alredy_in !=ans_candidate[row][i]:
                    flag_row_common_sense =True
                    ans_candidate[row][i] =ans_candidate[row][i] - board_alredy_in
    return flag_row_common_sense
               
#列の常識               
def colum_common_sense(ans_candidate,board):
    flag_colum_common_sense =False
    #各列の数字を取得
    for colum in range(9):
        board_alredy_in =set()
        for row in range(9):
            if board[row][colum]!=" ":
                board_alredy_in.add(board[row][colum])
        #列の常識を使い、候補を絞り込む
        for i in range(9):
            if board[i][colum] ==" ":
                if ans_candidate[i][colum] - board_alredy_in !=ans_candidate[i][colum]:
                    flag_colum_common_sense =True
                    ans_candidate[i][colum] = ans_candidate[i][colum] - board_alredy_in
                   
    return flag_colum_common_sense

#ブロックの常識               
def block_common_sense(ans_candidate,board):
    flag_block_common_sense =False
    #各ブロックの数字を取得
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        board_alredy_in =set()
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum]!=" ":
                    board_alredy_in.add(board[row+block_row][colum+block_colum])
        
        #ブロックの常識を使い、候補を絞り込む
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum] ==" ":
                    if ans_candidate[row+block_row][colum+block_colum] - board_alredy_in !=ans_candidate[row+block_row][colum+block_colum]:
                        flag_block_common_sense =True
                        ans_candidate[row+block_row][colum+block_colum] =ans_candidate[row+block_row][colum+block_colum] - board_alredy_in
    return flag_block_common_sense    
    



# プログラムが正しく動いているか？入力値に不備がないか？

In [3]:
def debug_board(board,input_data):
    teacher_ans =last_means(input_data)#解く前のデータに最終手段を適用→模範解答
    for i in range(9):
        for j in range(9):
            if board[i][j] !=teacher_ans[i][j] and board[i][j] !=" ":
                print("座標",i,j)
                print(board,"作成データ")
                print(teacher_ans,"模範解答")
                return False
     
    return True
    

def def_test_board(board):
    test_board =list(board.reshape(1,81)[0])
    for i in range(81):
        if test_board[i] ==" ":
            test_board[i] =str(0)
    test_board ="".join(test_board)
    return test_board
    
    
def debug_ans_candidate(ans_candidate,input_data,board):
    teacher_ans =last_means(input_data)
    for i in range(9):
        for j in range(9):
            if (str(teacher_ans[i][j]) not in ans_candidate[i][j]) and board[i][j]==" ":
                print(i,j)
                return False
    return True

def output(level,input_data):
    for i in range(81): #入力データが正常かどうか判断
        if input_data[i] ==" ":
            level ="error"
            return level  
    for i in range(9):
        for j in range(9):
            if board[i][j] ==" ":
                level =0
                return level
    return level


# それぞれの解法

In [4]:
#一ヶ所確定   
def one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number): 
    row_alredy_in =set() #着目している数字がある行
    colum_alredy_in =set() #着目している数字がある列
    block_alredy_in =set() #着目している数字があるブロック
    block_number =0 #ブロックに番号を振る
        
    #着目している数字の探索、入らない場所の特定
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum] ==" ":continue
                if int(board[row+block_row][colum+block_colum]) ==target_number:
                    row_alredy_in.add(row+block_row)
                    colum_alredy_in.add(colum+block_colum)
                    block_alredy_in.add(block_number)
        block_number +=1
    return row_alredy_in,colum_alredy_in,block_alredy_in






def one_place_confirmed_block(ans_candidate,board,target_point_row,target_point_colum):#着目している数字(target_number)が、あるブロックにおいて1度のみしか現れないかどうか判定する  
    flag_one_place_confirmed_block =False
    for target_number in range(1,10):
        row_alredy_in,colum_alredy_in,block_alredy_in =one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number)#(target_number:着目する数字) target_numberが入らない座標
        can_fill_in_block =[] #ブロックにおいて着目した場合に数字を埋めることができる座標
        block_number =0
        for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
            block_row =block[0]
            block_colum =block[1]
            if block_number in block_alredy_in:
                block_number +=1
                continue
            #ブロックにおいて着目した場合に数字を埋めることができる座標が１つの時にboardを更新する・        
            if len(can_fill_in_block) ==1:
                if target_point_row != can_fill_in_block[0][0]  or target_point_colum != can_fill_in_block[0][1]:continue
                board[can_fill_in_block[0][0]][can_fill_in_block[0][1]] =str(target_number)
                flag_one_place_confirmed_block =True
            can_fill_in_block =[]
            for row in range(3):
                for colum in range(3):
                    if row+block_row in row_alredy_in:
                       break
                    if board[row+block_row][colum+block_colum] !=" " or  colum+block_colum in colum_alredy_in:continue
                    #着目している数字があるブロックにおいて1度のみ入れることが可能かどうか判定する    
                    if len(can_fill_in_block) ==0:
                        can_fill_in_block.append([row+block_row,colum+block_colum])
                    else:
                        can_fill_in_block.append("damy")
                        break
            block_number +=1
    return flag_one_place_confirmed_block
               
               
def one_place_confirmed_row(ans_candidate,board,target_point_row,target_point_colum ):#着目している数字(target_number)が、ある行において1度のみしか現れないかどうか判定する  
    flag_one_place_confirmed_row =False
    for target_number in range(1,10):
        row_alredy_in,colum_alredy_in,block_alredy_in =one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number)#target_number:着目する数字
        can_fill_in_row =[] #行において着目した場合に数字を埋めることができる座標
        for row in range(9):
            if len(can_fill_in_row) ==1:#行において着目した場合に数字を埋めることができる座標が１つの時にboardを更新する・
                if target_point_row !=can_fill_in_row[0][0]   or target_point_colum !=can_fill_in_row[0][1]:continue   
                board[can_fill_in_row[0][0]][can_fill_in_row[0][1]] =str(target_number)
                flag_one_place_confirmed_row =True
            can_fill_in_row =[]
            for colum in range(9):
                if row in row_alredy_in:
                    break
                if board[row][colum] !=" " or  colum in colum_alredy_in:continue
                #着目している数字がある行において1度のみ入れることが可能かどうか判定する  
                if len(can_fill_in_row) ==0:
                    can_fill_in_row.append([row,colum])
                else:
                    can_fill_in_row.append("damy")
                    break
    return flag_one_place_confirmed_row
                
        
        

def one_place_confirmed_colum(ans_candidate,board,target_point_row,target_point_colum):#着目している数字(target_number)が、ある列において1度のみしか現れないかどうか判定する  
    flag_one_place_confirmed_colum =False
    for target_number in range(1,10):
        row_alredy_in,colum_alredy_in,block_alredy_in =one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number)#target_number:着目する数字
        can_fill_in_colum =[] #列において着目した場合に数字を埋めることができる座標
        for colum in range(9):
            if len(can_fill_in_colum) ==1:#列において着目した場合に数字を埋めることができる座標が１つの時にboardを更新する・ 
                if target_point_row !=can_fill_in_colum[0][0]   or target_point_colum!=can_fill_in_colum[0][1]:continue 
                board[can_fill_in_colum[0][0]][can_fill_in_colum[0][1]] =str(target_number)
                flag_one_place_confirmed_colum =True
            can_fill_in_colum =[]
            if colum in colum_alredy_in:continue
            for row in range(9):
                if board[row][colum] !=" " or  row in row_alredy_in:continue
                if len(can_fill_in_colum) ==0:#着目している数字がある列において1度のみ入れることが可能かどうか判定する  
                    can_fill_in_colum.append([row,colum])
                else:
                    can_fill_in_colum.append("damy")
                    break
    return flag_one_place_confirmed_colum
       
       
#二国同盟/定員確定法(ブロックに着目)
def two_country_block(ans_candidate,board,target_point_row,target_point_colum ):
    flag_two_country_block =False
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        
        two_ans_candidate =[] #答えの候補が２つのもの
        del_ans_candidate =[] #ブロック内の候補から削除できる数字
        
        two_ans_candidate_point =[] #答えの候補が２つのものの座標
        del_ans_candidate_point =[] #削除できる候補の数字がある座標(削除しない座標)
        for row in range(3):
            for colum in range(3):
                if len(list(ans_candidate[row+block_row][colum+block_colum])) ==2 and board[row+block_row][colum+block_colum] ==" ":#答えの候補が２つのものを判断する
                    two_ans_candidate.append(copy.copy(ans_candidate[row+block_row][colum+block_colum]))
                    two_ans_candidate_point.append([row+block_row,colum+block_colum])
                    for i in range(len(two_ans_candidate)-1):#答えの候補が２つであり同一のものを探索する
                        if two_ans_candidate[i] ==ans_candidate[row+block_row][colum+block_colum]:
                            del_ans_candidate.append(two_ans_candidate[i]) #削除できる候補をdel_ans_candidateに保存する
                            del two_ans_candidate[i]
                            del two_ans_candidate[-1]
                            
                            del_ans_candidate_point.append([two_ans_candidate_point[i],two_ans_candidate_point[-1]])
                            del two_ans_candidate_point[i]
                            del two_ans_candidate_point[-1]
                            break
                        
                            
        #削除できる候補をans_candidateから削除する
        for i in range(len(del_ans_candidate)):
            for j in range(2):
                del_number =str(del_ans_candidate[i].pop()) #削除する数字
                for row in range(3):
                    for colum in range(3):
                        if (del_ans_candidate_point[i][0][0] ==row+block_row and del_ans_candidate_point[i][0][1] ==colum+block_colum) or (del_ans_candidate_point[i][1][0] ==row+block_row and del_ans_candidate_point[i][1][1] ==colum+block_colum):continue
                        if (del_number in ans_candidate[row+block_row][colum+block_colum]):
                            if target_point_row !=row+block_row   or target_point_colum !=colum+block_colum:continue 
                            ans_candidate[row+block_row][colum+block_colum].remove(del_number)
                            flag_two_country_block =True
            
    return flag_two_country_block
    
#二国同盟/定員確定法(行に着目)
def two_country_row(ans_candidate,board,target_point_row,target_point_colum):
    flag_two_country_row =False
    for row in range(9):
        two_ans_candidate =[] #答えの候補が２つのもの
        del_ans_candidate =[] #探査中の行の候補から削除できる数字
        
        two_ans_candidate_point =[] #答えの候補が２つのものの座標
        del_ans_candidate_point =[] #削除できる候補の数字がある座標(削除しない座標)
        for colum in range(9):
            if len(list(ans_candidate[row][colum])) ==2 and board[row][colum] ==" ":#答えの候補が２つのものを判断する
                    two_ans_candidate.append(copy.copy(ans_candidate[row][colum]))
                    two_ans_candidate_point.append([row,colum])
                    for i in range(len(two_ans_candidate)-1):#答えの候補が２つであり同一のものを探索する
                        if two_ans_candidate[i] ==ans_candidate[row][colum]:
                            del_ans_candidate.append(two_ans_candidate[i]) #削除できる候補をdel_ans_candidateに保存する
                            del two_ans_candidate[i]
                            del two_ans_candidate[-1]
                            
                            del_ans_candidate_point.append([two_ans_candidate_point[i],two_ans_candidate_point[-1]])
                            del two_ans_candidate_point[i]
                            del two_ans_candidate_point[-1]
                            break
               
        #削除できる候補をans_candidateから削除する
        for i in range(len(del_ans_candidate)):
            for j in range(2):
                del_number =str(del_ans_candidate[i].pop()) #削除する数字
                for colum in range(9):
                    if (del_ans_candidate_point[i][0][0] ==row and del_ans_candidate_point[i][0][1] ==colum) or (del_ans_candidate_point[i][1][0] ==row and del_ans_candidate_point[i][1][1] ==colum):continue
                    if (del_number in ans_candidate[row][colum]):
                        if target_point_row!=row   or target_point_colum!=colum:continue 
                        ans_candidate[row][colum].remove(del_number)
                        flag_two_country_row =True
    return flag_two_country_row
    
#二国同盟/定員確定法(列に着目)
def two_country_colum(ans_candidate,board,target_point_row,target_point_colum ):
    flag_two_country_colum =False
    for colum in range(9):
        two_ans_candidate =[] #答えの候補が２つのもの
        del_ans_candidate =[] #探査中の行の候補から削除できる数字
        
        two_ans_candidate_point =[] #答えの候補が２つのものの座標
        del_ans_candidate_point =[] #削除できる候補の数字がある座標(削除しない座標)
        for row in range(9):
            if len(list(ans_candidate[row][colum])) ==2 and board[row][colum] ==" ":#答えの候補が２つのものを判断する
                    two_ans_candidate.append(copy.copy(ans_candidate[row][colum]))
                    two_ans_candidate_point.append([row,colum])
                    for i in range(len(two_ans_candidate)-1):#答えの候補が２つであり同一のものを探索する
                        if two_ans_candidate[i] ==ans_candidate[row][colum]:
                            del_ans_candidate.append(two_ans_candidate[i]) #削除できる候補をdel_ans_candidateに保存する
                            del two_ans_candidate[i]
                            del two_ans_candidate[-1]
                            
                            del_ans_candidate_point.append([two_ans_candidate_point[i],two_ans_candidate_point[-1]])
                            del two_ans_candidate_point[i]
                            del two_ans_candidate_point[-1]
                            break
               
        #削除できる候補をans_candidateから削除する
        for i in range(len(del_ans_candidate)):
            for j in range(2):
                del_number =str(del_ans_candidate[i].pop()) #削除する数字
                for row in range(9):
                    if (del_ans_candidate_point[i][0][0] ==row and del_ans_candidate_point[i][0][1] ==colum) or (del_ans_candidate_point[i][1][0] ==row and del_ans_candidate_point[i][1][1] ==colum):continue
                    if (del_number in ans_candidate[row][colum]):
                        if target_point_row !=row  or target_point_colum !=colum:continue 
                        ans_candidate[row][colum].remove(del_number)
                        flag_two_country_colum =True
    return flag_two_country_colum
 
    
    
    
#三国同盟/定員確定法(ブロックに着目)
def three_country_block(ans_candidate,board,target_point_row,target_point_colum ):
    three_country_block =False
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        
        three_ans_candidate =[] #答えの候補が3つのもの
        del_ans_candidate =[] #ブロック内の候補から削除できる数字
        
        for row in range(3):
            for colum in range(3):
                if len(list(ans_candidate[row+block_row][colum+block_colum])) ==3 and board[row+block_row][colum+block_colum] ==" ":#答えの候補が3つのものを判断する
                    three_ans_candidate.append(copy.copy(ans_candidate[row+block_row][colum+block_colum]))

        combination_three_ans_candidate = itertools.combinations(three_ans_candidate, 3)
        for com in combination_three_ans_candidate:#答えの候補が3つであり同一のものを探索する
            if com[0] ==com[1] and com[1] ==com[2] and com[0] ==com[2]:
                del_ans_candidate.append(com[0])

        #削除できる候補をans_candidateから削除する
        for i in range(len(del_ans_candidate)):
            del_target =del_ans_candidate[i]
            del_target_store =set([])
            for row in range(3):
                for colum in range(3):
                    if ans_candidate[row+block_row][colum+block_colum] ==del_target:continue
                    for j in range(3):
                        del_number =str(del_target.pop())
                        del_target_store.add(del_number)
                        if del_number in ans_candidate[row+block_row][colum+block_colum]:
                            if target_point_row !=row+block_row   or target_point_colum !=colum+block_colum:continue 
                            ans_candidate[row+block_row][colum+block_colum].remove(del_number)
                            three_country_block =True
                    del_target =del_target_store
    return three_country_block
    
#三国同盟/定員確定法(行に着目)
def three_country_row(ans_candidate,board,target_point_row,target_point_colum ):
    three_country_row =False
    for row in range(9):
        three_ans_candidate =[] #答えの候補が3つのもの
        del_ans_candidate =[] #探査中の行の候補から削除できる数字        
        for colum in range(9):
            if len(list(ans_candidate[row][colum])) ==3 and board[row][colum] ==" ":#答えの候補が3つのものを判断する
                    three_ans_candidate.append(copy.copy(ans_candidate[row][colum]))
        combination_three_ans_candidate = itertools.combinations(three_ans_candidate, 3)
        for com in combination_three_ans_candidate:#答えの候補が3つであり同一のものを探索する
            if com[0] ==com[1] and com[1] ==com[2] and com[0] ==com[2]:
                del_ans_candidate.append(com[0])
        
        for i in range(len(del_ans_candidate)):
            del_target =del_ans_candidate[i]
            del_target_store =set([])
            for colum in range(9):
                if ans_candidate[row][colum] ==del_target:continue
                for j in range(3):
                    del_number =str(del_target.pop())
                    del_target_store.add(del_number)
                    if del_number in ans_candidate[row][colum]:
                        if target_point_row!=row   or target_point_colum !=colum:continue 
                        ans_candidate[row][colum].remove(del_number)
                        three_country_row =True
                del_target =del_target_store
        
    return three_country_row
    
#三国同盟/定員確定法(列に着目)
def three_country_colum(ans_candidate,board,target_point_row,target_point_colum ):
    three_country_colum =False
    for colum in range(9):
        three_ans_candidate =[] #答えの候補が3つのもの
        del_ans_candidate =[] #探査中の行の候補から削除できる数字
        for row in range(9):
            if len(list(ans_candidate[row][colum])) ==3 and board[row][colum] ==" ":#答えの候補が3つのものを判断する
                three_ans_candidate.append(copy.copy(ans_candidate[row][colum]))
        combination_three_ans_candidate = itertools.combinations(three_ans_candidate, 3)
        for com in combination_three_ans_candidate:#答えの候補が3つであり同一のものを探索する
            if com[0] ==com[1] and com[1] ==com[2] and com[0] ==com[2]:
                del_ans_candidate.append(com[0])
        
        for i in range(len(del_ans_candidate)):
            del_target =del_ans_candidate[i]
            del_target_store =set([])
            for row in range(9):
                if ans_candidate[row][colum] ==del_target:continue
                for j in range(3):
                    del_number =str(del_target.pop())
                    del_target_store.add(del_number)
                    if del_number in ans_candidate[row][colum]:
                        if target_point_row !=row  or target_point_colum!=colum:continue 
                        ans_candidate[row][colum].remove(del_number)
                        three_country_colum =True
                del_target =del_target_store
    return three_country_colum
    
    

    
    
    
    
    


#四角対角線(行に着目)
def square_diagonal_row(ans_candidate,board,target_point_row,target_point_colum ):
    flag_square_diagonal_row =False
    for target_number in range(1,10):#target_numberは着目する数字
        two_ans_candidate_target_number =[]#target_numberが入る候補が２つの座標を保存する
        for row in range(9):
            row_target_number_cnt =0#ある行に関してtraget_numberが入る可能性がある箇所の個数を記録する
            target_number_in_point =[row] #ある行に関してtraget_numberが入る可能性がある箇所の[行の座標]「列の座標」を保存する→[行の座標,列の座標（1つ目）,列の座標（2つ目）]
            for colum in range(9): #ある行に関してtraget_numberが入る可能性がある箇所の個数を求める
                if str(target_number) in ans_candidate[row][colum]:
                    row_target_number_cnt +=1
                    target_number_in_point.append(colum)
                    if row_target_number_cnt ==4: #traget_numberが入る可能性がある箇所が３つ以上であれば四角対角線は使えない
                        break
            if row_target_number_cnt ==2:#ある行に関してtraget_numberが入る可能性がある箇所の個数が２つの時、四角対角線を使える可能性がある
                two_ans_candidate_target_number.append(target_number_in_point)
       
        
        for search in  itertools.combinations(two_ans_candidate_target_number, 2):#候補のそれぞれの列が２つとも一致する行の組み合わせを探す。

            del_col =[]#target_numberを候補から消せる列
            if search[0][1] ==search[1][1] and search[0][2] ==search[1][2]:
                del_col.append(search[0][1])
                del_col.append(search[0][2])
                for colum in del_col:
                    for row in range(9):
                        if (str(target_number) in ans_candidate[row][colum]):
                            if search[0][0] ==row or search[1][0] ==row:continue
                            if target_point_row !=row  or target_point_colum!=colum:continue 
                            ans_candidate[row][colum].remove(str(target_number))
                            flag_square_diagonal_row =True
                        
    return flag_square_diagonal_row

#四角対角線(列に着目)
def square_diagonal_colum(ans_candidate,board,target_point_row,target_point_colum ):
    flag_square_diagonal_colum =False
    for target_number in range(1,10):#target_numberは着目する数字
        two_ans_candidate_target_number =[]#target_numberが入る候補が２つの座標を保存する
        for colum in range(9):
            row_target_number_cnt =0#ある列に関してtraget_numberが入る可能性がある箇所の個数を記録する
            target_number_in_point =[colum] #ある列に関してtraget_numberが入る可能性がある箇所の[行の座標]「列の座標」を保存する→[列の座標,行の座標（1つ目）,行の座標（2つ目）]
            for row in range(9): #ある行に関してtraget_numberが入る可能性がある箇所の個数を求める
                if str(target_number) in ans_candidate[row][colum]:
                    row_target_number_cnt +=1
                    target_number_in_point.append(row)
                    if row_target_number_cnt ==4: #traget_numberが入る可能性がある箇所が３つ以上であれば四角対角線は使えない
                        break
            if row_target_number_cnt ==2:#ある列に関してtraget_numberが入る可能性がある箇所の個数が２つの時、四角対角線を使える可能性がある
                two_ans_candidate_target_number.append(target_number_in_point)
       
        for search in  itertools.combinations(two_ans_candidate_target_number, 2):#候補のそれぞれの列が２つとも一致する行の組み合わせを探す。
            del_row =[]#target_numberを候補から消せる行
            if search[0][1] ==search[1][1] and search[0][2] ==search[1][2]:
                del_row.append(search[0][1])
                del_row.append(search[0][2])
                for row in del_row:
                    for colum in range(9):
                        if (str(target_number) in ans_candidate[row][colum]):
                            if search[0][0] ==colum or search[1][0] ==colum:continue
                            if target_point_row !=row  or target_point_colum !=colum:continue 
                            ans_candidate[row][colum].remove(str(target_number))
                            flag_square_diagonal_colum =True
                           
                         
    return flag_square_diagonal_colum


   
    
    
                       
#最終手段（全探査）

def fi(s): #次にどこのマスに数字を入れればいいか(入る候補が最も少ないのはどこか)を判定します
    _ = []
    used_yoko = [set([s[k%9+l*9] for l in range(9)]) for k in range(9)] #あらかじめ、各列各行各ブロックで使用済みの数字をリストアップしておきます
    used_tate = [set([s[k*9+l] for l in range(9)]) for k in range(9)]
    used_block = [set([s[k//3*27+k%3*3+l//3*9+l%3] for l in range(9)]) for k in range(9)]
    for k in range(81):
      if s[k] != "0":
        _ += [0]
      else:
        ss = used_yoko[k%9]|used_tate[k//9]|used_block[k//27*3+k%9//3]
        if "0" in ss:
          ss.remove("0")
        _ += [len(set(ss))]
    k_next = _.index(max(_))
    ss_next =used_yoko[k_next%9]|used_tate[k_next//9]|used_block[k_next//27*3+k_next%9//3]
    return k_next,set(map(str,range(1,10)))-set(ss_next)

def dfs(i,t): #全探索を行う部分です
  global board_last_means
  if i == sudoku.count("0"):
    board_last_means =np.array(list(t)).reshape(9,9)
  else:
    k,next_number = fi(t)
    for j in next_number:
      dfs(i+1,t[:k]+j+t[k+1:])
    
def last_means(data):#全探査をしたか判定
    global sudoku
    sudoku = data 
    dfs(0,sudoku)
    return board_last_means



  



# 座標の優先度を特定

In [5]:

def few_ans_candidate(ans_candidate,board,no_search_point):#それぞれの候補と候補の最小数を求める
    few_point =[float("inf"),float("inf")]
    ans_quantity_min =float("inf")
    ans_quantity_array =[]
    
    bury =0 #数字が埋まっている座標の個数
    cant =0 #空白だが、解法を適用出来なかった座標の個数
    out_point =[]
    
    for row in range(9):
        quantity_row =[]
        for colum in range(9):
            if board[row][colum] !=" ":
                quantity_row.append(float("inf"))
                out_point.append(row*9+colum)
                bury +=1
                if len(out_point) ==81:
                    return [-1,-1]
                continue  
            if row*9+colum in no_search_point:
                quantity_row.append(float("inf"))
                out_point.append(row*9+colum)
                cant +=1
                if len(out_point) ==81:
                    return [-1,-1]
                continue 
            quantity_row.append(len(ans_candidate[row][colum]))
            ans_quantity_min =min(ans_quantity_min,len(ans_candidate[row][colum]))
            if ans_quantity_min ==float("inf"):
                return [-1,-1]
        ans_quantity_array.append(quantity_row)
        
    ans_quantity_min_point =[]
    for row in range(9):
        for colum in range(9):
            if ans_quantity_array[row][colum] ==ans_quantity_min:
                ans_quantity_min_point.append([row,colum])
    return ans_quantity_min_point #候補数が最小の座標

def def_bury_row(board):
    bury_row =[]
    for row in range(9):
        pre =0
        for colum in range(9):
            if board[row][colum] !=" ":
                pre +=1
        bury_row.append(pre)
    return bury_row

def def_bury_colum(board):
    bury_colum =[]
    for colum in range(9):
        pre =0
        for row in range(9):
            if board[row][colum] !=" ":
                pre +=1
        bury_colum.append(pre)
    return bury_colum

def def_bury_block(board):
    bury_block =[]
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        pre =0
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum]!=" ":
                    pre +=1
        bury_block.append(pre)
    return bury_block

def def_priority(board): #優先度=100×ブロック内で埋まっている数字+列内で埋まっている数字+行内で埋まっている数字
    bury_row = def_bury_row(board)
    bury_colum = def_bury_colum(board)
    bury_block = def_bury_block(board)
    priority =[[0 for _ in range(9)]for _ in range(9)]
    block_number =0
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum] !=" ":continue
                p = 100*bury_block[block_number]+bury_row[row+block_row]+bury_colum[colum+block_colum]
                priority[row+block_row][colum+block_colum] =p
        block_number +=1
    return priority

def few_ans_candidate_and_hi_priority(ans_candidate,board,no_search_point): #候補数が少ない内で優先度が最も高いもの(全て同じであればランダムに抽出する)
    ans_quantity_min_point =few_ans_candidate(ans_candidate,board,no_search_point)
    if ans_quantity_min_point ==[-1,-1]:
        return -1,-1
    priority =def_priority(board)
    pr =0
    few_row =[]
    few_colum =[]
    for i in range(len(ans_quantity_min_point)):
        if pr < priority[ans_quantity_min_point[i][0]][ans_quantity_min_point[i][1]]:
            few_row =[]
            few_colum =[]
            pr = priority[ans_quantity_min_point[i][0]][ans_quantity_min_point[i][1]]
            few_row.append(ans_quantity_min_point[i][0])
            few_colum.append(ans_quantity_min_point[i][1])
        if  pr == priority[ans_quantity_min_point[i][0]][ans_quantity_min_point[i][1]]:
            few_row.append(ans_quantity_min_point[i][0])
            few_colum.append(ans_quantity_min_point[i][1])
    r =random.randrange(len(few_row)) 
    row,colum =few_row[r],few_colum[r]
    return row,colum

# 列の常識、行の常識、ブロックの常識、残り物の常識の使用回数カウント

In [6]:
def row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board):
    flag =False
    if leftovers(ans_candidate,board):
        use_leftovers +=1
        flag =True
    if row_common_sense(ans_candidate,board):
        use_row +=1
        flag =True
    if colum_common_sense(ans_candidate,board):
        use_colum +=1
        flag =True
    if block_common_sense(ans_candidate,board):
        use_block +=1
        flag =True
    return use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag
    

# 分類

In [26]:
def classify_few(difficulty,blank_count_array,common_sense_average,very_very_easy_cnt,very_easy_cnt,easy_cnt,cant_solve):
    #if common_sense_average<2.36205 and difficulty<0.01: #簡単すぎと超簡単の分類(常識の使用回数)
    #    very_very_easy_cnt +=1
    #else:
    #   very_easy_cnt +=1
    if difficulty=="解けない":
        cant_solve +=1
        return  very_very_easy_cnt,very_easy_cnt,easy_cnt,cant_solve
        
    if  difficulty<0.01: #簡単すぎと超簡単の分類(空欄の個数)
        if blank_count_array[number]<33:
            very_very_easy_cnt +=1
            print("asasfa")
        else:
            very_easy_cnt +=1
    elif difficulty<3.5:
        very_easy_cnt +=1
    elif difficulty<11.4:
        easy_cnt +=1
    return very_very_easy_cnt,very_easy_cnt,easy_cnt,cant_solve

# 深さ優先探索（候補が少ないかつ周囲が埋まっているものから埋める）

In [27]:
def execution_few(ans_candidate,board,blank_count_array):#候補が少ないかつ周囲が埋まっているものから埋める(深さ優先探索)    
    used =[0 for _ in range(8)]
    flag_board_complete =False
    first_search =True
    no_search_point =set() #着目したが、候補数を減らすことが出来なかった座標
    use_leftovers =0 #残り物の常識を使用した回数
    use_row =0 #行の常識を使用した回数
    use_colum =0#列の常識を使用した回数
    use_block =0#ブロックの常識を使用した回数
    while True:
        flag_candidate_change =False
        if finish(board):
            flag_board_complete =True
            break
        start_ans_candidate =copy.deepcopy(ans_candidate)
        start_board =copy.deepcopy(board)
        
    
        target_point_row,target_point_colum =few_ans_candidate_and_hi_priority(ans_candidate,board,no_search_point)
        if target_point_row ==-1:
            break 

        use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
        
        if  one_place_confirmed_row(ans_candidate,board,target_point_row,target_point_colum ) or one_place_confirmed_colum(ans_candidate,board,target_point_row,target_point_colum )  or one_place_confirmed_block(ans_candidate,board,target_point_row,target_point_colum ):
            used[1] +=1
            use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
            flag_candidate_change =True
            
        if  two_country_row(ans_candidate,board,target_point_row,target_point_colum ) or two_country_colum(ans_candidate,board,target_point_row,target_point_colum )  or two_country_block(ans_candidate,board,target_point_row,target_point_colum ):
            used[2] +=1
            use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
            flag_candidate_change =True
                    
        if  three_country_block(ans_candidate,board,target_point_row,target_point_colum ) or three_country_row(ans_candidate,board,target_point_row,target_point_colum ) or three_country_colum(ans_candidate,board,target_point_row,target_point_colum ):  
            used[3] +=1
            use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
            flag_candidate_change =True
                    
        if  square_diagonal_row(ans_candidate,board,target_point_row,target_point_colum ) or square_diagonal_colum(ans_candidate,board,target_point_row,target_point_colum ):
            used[4] +=1
            use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
            flag_candidate_change =True
        
        
        if flag_candidate_change is False:
            no_search_point.add(target_point_row*9+target_point_colum)
        else:
            no_search_point =set()
           
            
    difficulty =0
    for i in range(8):  #レベルの計算
        difficulty +=used[i]*(i)
    if flag_board_complete is False: #盤面が完成していない場合
        difficulty ="解けない"
    common_sense_average =(use_leftovers + use_colum + use_row + use_block)/4
    
    return difficulty ,use_leftovers,use_colum,use_row,use_block,common_sense_average
        
                    
    




n =int(input())
data_input(n)
difficulty_array =[]


store_board = []
for number in range(n):
    store_board.append(copy.deepcopy(board_all[number]))


use_leftovers_sum =0 #残り物の常識を使用した回数の合計
use_colum_sum =0 #列の常識を使用した回数の合計
use_row_sum =0 #行の常識を使用した回数の合計
use_block_sum =0 #ブロックの常識を使用した回数の合計

very_very_easy_cnt =0 #簡単すぎ
very_easy_cnt =0 #超簡単
easy_cnt =0 #簡単

cant_solve =0#解けない





#候補が少ない個所から埋める
print("候補が少ない個所から埋める")
for number in range(n):
    board =copy.deepcopy(store_board[number])
    board =np.array(list(board)).reshape(9,9)
    

    
    ans_candidate =ans_candidate_set(board)
    difficulty,use_leftovers,use_colum,use_row,use_block,common_sense_average =execution_few(ans_candidate,board,blank_count_array)
    use_leftovers_sum +=use_leftovers
    use_colum_sum +=use_colum_sum
    use_row_sum +=use_row
    use_block_sum +=use_block
    
    
    input_comfirm = output(difficulty,input_data[number])
    if difficulty =="error":
        print(number,"error")
        break
    else:
        difficulty_array.append(difficulty)
        
    print(number+1,"番目の入力",":","難易度",difficulty,end="　")  
    if debug_board(board,input_data[number]) is True:
        print("プログラムは正常です。")
    else:
        print(input_data[number],"以上の入力データにエラー")
        break
        
    very_very_easy_cnt,very_easy_cnt,easy_cnt,cant_solve =classify_few(difficulty,blank_count_array,common_sense_average,very_very_easy_cnt,very_easy_cnt,easy_cnt,cant_solve)
        

print("簡単すぎ",very_very_easy_cnt,"超簡単",very_easy_cnt,"簡単",easy_cnt)



1000
...


候補が少ない個所から埋める
1 番目の入力 : 難易度 3　プログラムは正常です。
2 番目の入力 : 難易度 2　プログラムは正常です。
3 番目の入力 : 難易度 3　プログラムは正常です。
4 番目の入力 : 難易度 10　プログラムは正常です。
5 番目の入力 : 難易度 17　プログラムは正常です。
6 番目の入力 : 難易度 2　プログラムは正常です。
7 番目の入力 : 難易度 12　プログラムは正常です。
8 番目の入力 : 難易度 8　プログラムは正常です。
9 番目の入力 : 難易度 4　プログラムは正常です。
10 番目の入力 : 難易度 8　プログラムは正常です。
11 番目の入力 : 難易度 1　プログラムは正常です。
12 番目の入力 : 難易度 12　プログラムは正常です。
13 番目の入力 : 難易度 7　プログラムは正常です。
14 番目の入力 : 難易度 2　プログラムは正常です。
15 番目の入力 : 難易度 11　プログラムは正常です。
16 番目の入力 : 難易度 19　プログラムは正常です。
17 番目の入力 : 難易度 5　プログラムは正常です。
18 番目の入力 : 難易度 12　プログラムは正常です。
19 番目の入力 : 難易度 4　プログラムは正常です。
20 番目の入力 : 難易度 8　プログラムは正常です。
21 番目の入力 : 難易度 3　プログラムは正常です。
22 番目の入力 : 難易度 4　プログラムは正常です。
23 番目の入力 : 難易度 10　プログラムは正常です。
24 番目の入力 : 難易度 9　プログラムは正常です。
25 番目の入力 : 難易度 8　プログラムは正常です。
26 番目の入力 : 難易度 3　プログラムは正常です。
27 番目の入力 : 難易度 3　プログラムは正常です。
28 番目の入力 : 難易度 22　プログラムは正常です。
29 番目の入力 : 難易度 21　プログラムは正常です。
30 番目の入力 : 難易度 10　プログラムは正常です。
31 番目の入力 : 難易度 1　プログラムは正常です。
32 番目の入力 : 難易度 10　プログラムは正常です。
33 番目の入力 : 難易度 19　プログラムは正常です。
34 番目の入力 : 難易度 3　プログラムは正常

276 番目の入力 : 難易度 12　プログラムは正常です。
277 番目の入力 : 難易度 7　プログラムは正常です。
278 番目の入力 : 難易度 4　プログラムは正常です。
279 番目の入力 : 難易度 2　プログラムは正常です。
280 番目の入力 : 難易度 2　プログラムは正常です。
281 番目の入力 : 難易度 18　プログラムは正常です。
282 番目の入力 : 難易度 8　プログラムは正常です。
283 番目の入力 : 難易度 1　プログラムは正常です。
284 番目の入力 : 難易度 8　プログラムは正常です。
285 番目の入力 : 難易度 5　プログラムは正常です。
286 番目の入力 : 難易度 6　プログラムは正常です。
287 番目の入力 : 難易度 12　プログラムは正常です。
288 番目の入力 : 難易度 5　プログラムは正常です。
289 番目の入力 : 難易度 5　プログラムは正常です。
290 番目の入力 : 難易度 2　プログラムは正常です。
291 番目の入力 : 難易度 8　プログラムは正常です。
292 番目の入力 : 難易度 4　プログラムは正常です。
293 番目の入力 : 難易度 2　プログラムは正常です。
294 番目の入力 : 難易度 18　プログラムは正常です。
295 番目の入力 : 難易度 19　プログラムは正常です。
296 番目の入力 : 難易度 2　プログラムは正常です。
297 番目の入力 : 難易度 4　プログラムは正常です。
298 番目の入力 : 難易度 6　プログラムは正常です。
299 番目の入力 : 難易度 2　プログラムは正常です。
300 番目の入力 : 難易度 13　プログラムは正常です。
301 番目の入力 : 難易度 3　プログラムは正常です。
302 番目の入力 : 難易度 21　プログラムは正常です。
303 番目の入力 : 難易度 3　プログラムは正常です。
304 番目の入力 : 難易度 4　プログラムは正常です。
305 番目の入力 : 難易度 10　プログラムは正常です。
306 番目の入力 : 難易度 2　プログラムは正常です。
307 番目の入力 : 難易度 6　プログラムは正常です。
308 番目の入力 : 難易度 2　プログラムは正常です。
30

549 番目の入力 : 難易度 21　プログラムは正常です。
550 番目の入力 : 難易度 1　プログラムは正常です。
551 番目の入力 : 難易度 2　プログラムは正常です。
552 番目の入力 : 難易度 0　プログラムは正常です。
553 番目の入力 : 難易度 1　プログラムは正常です。
554 番目の入力 : 難易度 14　プログラムは正常です。
555 番目の入力 : 難易度 4　プログラムは正常です。
556 番目の入力 : 難易度 6　プログラムは正常です。
557 番目の入力 : 難易度 10　プログラムは正常です。
558 番目の入力 : 難易度 0　プログラムは正常です。
559 番目の入力 : 難易度 3　プログラムは正常です。
560 番目の入力 : 難易度 3　プログラムは正常です。
561 番目の入力 : 難易度 15　プログラムは正常です。
562 番目の入力 : 難易度 2　プログラムは正常です。
563 番目の入力 : 難易度 7　プログラムは正常です。
564 番目の入力 : 難易度 4　プログラムは正常です。
565 番目の入力 : 難易度 7　プログラムは正常です。
566 番目の入力 : 難易度 2　プログラムは正常です。
567 番目の入力 : 難易度 10　プログラムは正常です。
568 番目の入力 : 難易度 1　プログラムは正常です。
569 番目の入力 : 難易度 7　プログラムは正常です。
570 番目の入力 : 難易度 12　プログラムは正常です。
571 番目の入力 : 難易度 2　プログラムは正常です。
572 番目の入力 : 難易度 1　プログラムは正常です。
573 番目の入力 : 難易度 11　プログラムは正常です。
574 番目の入力 : 難易度 8　プログラムは正常です。
575 番目の入力 : 難易度 1　プログラムは正常です。
576 番目の入力 : 難易度 7　プログラムは正常です。
577 番目の入力 : 難易度 9　プログラムは正常です。
578 番目の入力 : 難易度 37　プログラムは正常です。
579 番目の入力 : 難易度 16　プログラムは正常です。
580 番目の入力 : 難易度 22　プログラムは正常です。
581 番目の入力 : 難易度 2　プログラムは正常です。


821 番目の入力 : 難易度 9　プログラムは正常です。
822 番目の入力 : 難易度 3　プログラムは正常です。
823 番目の入力 : 難易度 1　プログラムは正常です。
824 番目の入力 : 難易度 4　プログラムは正常です。
825 番目の入力 : 難易度 10　プログラムは正常です。
826 番目の入力 : 難易度 7　プログラムは正常です。
827 番目の入力 : 難易度 1　プログラムは正常です。
828 番目の入力 : 難易度 6　プログラムは正常です。
829 番目の入力 : 難易度 10　プログラムは正常です。
830 番目の入力 : 難易度 3　プログラムは正常です。
831 番目の入力 : 難易度 2　プログラムは正常です。
832 番目の入力 : 難易度 7　プログラムは正常です。
833 番目の入力 : 難易度 5　プログラムは正常です。
834 番目の入力 : 難易度 32　プログラムは正常です。
835 番目の入力 : 難易度 9　プログラムは正常です。
836 番目の入力 : 難易度 17　プログラムは正常です。
837 番目の入力 : 難易度 2　プログラムは正常です。
838 番目の入力 : 難易度 1　プログラムは正常です。
839 番目の入力 : 難易度 20　プログラムは正常です。
840 番目の入力 : 難易度 3　プログラムは正常です。
841 番目の入力 : 難易度 10　プログラムは正常です。
842 番目の入力 : 難易度 4　プログラムは正常です。
843 番目の入力 : 難易度 6　プログラムは正常です。
844 番目の入力 : 難易度 4　プログラムは正常です。
845 番目の入力 : 難易度 2　プログラムは正常です。
846 番目の入力 : 難易度 6　プログラムは正常です。
847 番目の入力 : 難易度 2　プログラムは正常です。
848 番目の入力 : 難易度 12　プログラムは正常です。
849 番目の入力 : 難易度 1　プログラムは正常です。
850 番目の入力 : 難易度 3　プログラムは正常です。
851 番目の入力 : 難易度 8　プログラムは正常です。
852 番目の入力 : 難易度 5　プログラムは正常です。
853 番目の入力 : 難易度 解けない　プログラムは正常です。


In [ ]:
簡単の分析
簡単すぎ 0 超簡単 300 簡単 485

# 幅探査優先法(左上から右下へブロックに着目する場合は特殊)

In [ ]:

def execution_bfs(ans_candidate,board,level):#幅探査優先法(左上から右下へブロックに着目する場合は特殊)
    used =[0 for _ in range(8)]
    flag_board_complete =False
    c =5
    use_leftovers =0 #残り物の常識を使用した回数
    use_row =0 #行の常識を使用した回数
    use_colum =0#列の常識を使用した回数
    use_block =0#ブロックの常識を使用した回数
    while True:
        if finish(board):
            flag_board_complete =True
            break
        
        start_ans_candidate =copy.deepcopy(ans_candidate)

        for t_row in range(9):
            for t_colum in range(9):
                target_point_row,target_point_colum =t_row,t_colum
                
                flag_sb =True
                while flag_sb is True:
                    use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
                    if flag:
                        flag_sb =True
                    else:
                        flag_sb =False
                        
                if  one_place_confirmed_row(ans_candidate,board,target_point_row,target_point_colum ) or one_place_confirmed_colum(ans_candidate,board,target_point_row,target_point_colum )  or one_place_confirmed_block(ans_candidate,board,target_point_row,target_point_colum ):
                    used[1] +=1
                    use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
                    continue 
                    
                if  two_country_row(ans_candidate,board,target_point_row,target_point_colum ) or two_country_colum(ans_candidate,board,target_point_row,target_point_colum )  or two_country_block(ans_candidate,board,target_point_row,target_point_colum ):
                    used[2] +=1
                    use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
                    continue
                    
                if  three_country_block(ans_candidate,board,target_point_row,target_point_colum ) or three_country_row(ans_candidate,board,target_point_row,target_point_colum ) or three_country_colum(ans_candidate,board,target_point_row,target_point_colum ):  
                    used[3] +=1
                    use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
                    continue
                    
                if  square_diagonal_row(ans_candidate,board,target_point_row,target_point_colum ) or square_diagonal_colum(ans_candidate,board,target_point_row,target_point_colum ):
                    used[4] +=1
                    use_leftovers,use_row,use_colum,use_block,ans_candidate,board,flag =row_colum_block_left(use_leftovers,use_row,use_colum,use_block,ans_candidate,board)
                    continue
                    
        cnt =0
        for i in range(9):
            for j in range(9):
                if start_ans_candidate[i][j] ==ans_candidate[i][j]:
                    cnt +=1
        if cnt ==81:
            break
    difficulty =0
    for i in range(8):  #レベルの計算
        difficulty +=used[i]*(i)
    if flag_board_complete is False: #盤面が完成していない場合
        difficulty ="解けない"
    
    if used[c] >0:
        difficulty =float("inf")
    
    common_sense_average =(use_leftovers + use_row + use_block)/3
    
    return difficulty ,use_leftovers,use_colum,use_row,use_block,common_sense_average
   

n =int(input())
data_input(n)
difficulty_array =[]

level_average =0

store_board = []
for number in range(n):
    store_board.append(copy.deepcopy(board_all[number]))

very_easy_cnt =0 #超簡単
very_very_easy_cnt =0 #簡単すぎ






#幅探査優先法(左上から右下へブロックに着目する場合は特殊)
print("幅探査優先法")
for number in range(n):
    board =copy.deepcopy(store_board[number])
    board =np.array(list(board)).reshape(9,9)
    ans_candidate =ans_candidate_set(board)
    difficulty =0
    difficulty,use_leftovers,use_colum,use_row,use_block,common_sense_average =execution_bfs(ans_candidate,board,difficulty)    
    
    use_leftovers_sum +=use_leftovers
    use_colum_sum +=use_colum_sum
    use_row_sum +=use_row
    use_block_sum +=use_block
    
    
    if difficulty!="解けない":
        difficulty_array.append(difficulty)
    
    print(number+1,"番目の入力",":","難易度",difficulty,end="")  
   
   
    if debug_board(board,input_data[number]) is True:
        print("プログラムは正常です。")
    else:
        print(input_data[number],"以上の入力データにエラー")
        break
    
    if common_sense_average<2.36205 and difficulty<0.01: #簡単すぎと超簡単の分類(常識の使用回数)
        very_very_easy_cnt +=1
    else:
        very_easy_cnt +=1
        
    #if blank_count_array[number]<(39.509-25.5435)/2+25.5435 and difficulty<0.01: #簡単すぎと超簡単の分類(空欄の個数)
     #   very_very_easy_cnt +=1
    #else:
     #   very_easy_cnt +=1

if len(difficulty_array)==0:
    print("すべてのナンプレは解くことができませんでした。")

    
print(difficulty_array)

print("簡単すぎ",very_very_easy_cnt,"超簡単",very_easy_cnt)



In [ ]:
#作成中
#Sword Fish(行に着目)
def sword_fish_row(ans_candidate,board):
    sword_fish_row =False
    for target_number in range(1,10):#target_numberは着目する数字
        two_ans_candidate_target_number =[]#target_numberが入る候補が２つの座標を保存する
        for row in range(9):
            row_target_number_cnt =0#ある行に関してtraget_numberが入る可能性がある箇所の個数を記録する
            target_number_in_point =[row] #ある行に関してtraget_numberが入る可能性がある箇所の[行の座標]「列の座標」を保存する→[行の座標,列の座標（1つ目）,列の座標（2つ目）]
            for colum in range(9): #ある行に関してtraget_numberが入る可能性がある箇所の個数を求める
                if str(target_number) in ans_candidate[row][colum]:
                    row_target_number_cnt +=1
                    target_number_in_point.append(colum)
                    if row_target_number_cnt ==4: #traget_numberが入る可能性がある箇所が３つ以上であれば四角対角線は使えない
                        break
            if row_target_number_cnt ==2:#ある行に関してtraget_numberが入る可能性がある箇所の個数が２つの時、四角対角線を使える可能性がある
                two_ans_candidate_target_number.append(target_number_in_point)
       
        
        for search in  itertools.combinations(two_ans_candidate_target_number, 2):#候補のそれぞれの列が２つとも一致する行の組み合わせを探す。

            del_col =[]#target_numberを候補から消せる列
            if search[0][1] ==search[1][1] and search[0][2] ==search[1][2]:
                del_col.append(search[0][1])
                del_col.append(search[0][2])
                for colum in del_col:
                    for row in range(9):
                        if (str(target_number) in ans_candidate[row][colum]):
                            if search[0][0] ==row or search[1][0] ==row:continue
                            ans_candidate[row][colum].remove(str(target_number))
                            flag_square_diagonal_row =True
    return flag_square_diagonal_row

#Sword Fish(列に着目)
def sword_fish_colum(ans_candidate,board):
    flag_square_diagonal_colum =False
    for target_number in range(1,10):#target_numberは着目する数字
        two_ans_candidate_target_number =[]#target_numberが入る候補が２つの座標を保存する
        for colum in range(9):
            row_target_number_cnt =0#ある列に関してtraget_numberが入る可能性がある箇所の個数を記録する
            target_number_in_point =[colum] #ある列に関してtraget_numberが入る可能性がある箇所の[行の座標]「列の座標」を保存する→[列の座標,行の座標（1つ目）,行の座標（2つ目）]
            for row in range(9): #ある行に関してtraget_numberが入る可能性がある箇所の個数を求める
                if str(target_number) in ans_candidate[row][colum]:
                    row_target_number_cnt +=1
                    target_number_in_point.append(row)
                    if row_target_number_cnt ==4: #traget_numberが入る可能性がある箇所が３つ以上であれば四角対角線は使えない
                        break
            if row_target_number_cnt ==2:#ある列に関してtraget_numberが入る可能性がある箇所の個数が２つの時、四角対角線を使える可能性がある
                two_ans_candidate_target_number.append(target_number_in_point)
       
        for search in  itertools.combinations(two_ans_candidate_target_number, 2):#候補のそれぞれの列が２つとも一致する行の組み合わせを探す。
            del_row =[]#target_numberを候補から消せる行
            if search[0][1] ==search[1][1] and search[0][2] ==search[1][2]:
                del_row.append(search[0][1])
                del_row.append(search[0][2])
                for row in del_row:
                    for colum in range(9):
                        if (str(target_number) in ans_candidate[row][colum]):
                            if search[0][0] ==colum or search[1][0] ==colum:continue
                            ans_candidate[row][colum].remove(str(target_number))
                            flag_square_diagonal_colum =True

    return flag_square_diagonal_colum



# 

In [ ]:


print(board)

    

#浜田ロジック 作成中
def hamada_block_two(ans_candidate,board ): #ブロック内の空白が２つの座標を抽出 (ブロック内の行と列は異なる)
    block_number =0
    block_candidate_two =[]
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        block_candidate_two_point =[]
        blank =0 #あるブロックにおいて空白の個数をカウントする
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum] ==" ":
                    blank +=1
                    pre =[row+block_row,colum+block_colum]
                    block_candidate_two_point.append(pre)     
        if blank ==2 :
            if block_candidate_two_point[0][0] !=block_candidate_two_point[1][0] and block_candidate_two_point[0][1] !=block_candidate_two_point[1][1]:
                block_candidate_two.append(block_candidate_two_point)
        block_number +=1
    print(block_candidate_two)
    return block_candidate_two

target_point_row =1
target_point_colum =1


def df_blank_colum(ans_candidate,board):#ある列において空白の個数と座標
    blank_colum =[]
    for colum in range(9):
        cnt_b =0
        blank_colum_point =[]
        for row in range(9):
            if board[row][colum] ==" ":
                cnt_b +=1
                blank_colum_point.append([row,colum])
                
        blank_colum.append([cnt_b,blank_colum_point])#ある列においての（空白の個数、[空白の座標行、空白の座標列]）
    return blank_colum



def df_blank_row(ans_candidate,board):#ある行において空白の個数と座標
    blank_row =[]
    for row in range(9):
        cnt_b =0
        blank_row_point =[]
        for colum in range(9):
            if board[row][colum] ==" ":
                cnt_b +=1
                blank_row_point.append([row,colum])
                
        blank_row.append([cnt_b,blank_row_point])#ある列においての（空白の個数、[空白の座標行、空白の座標列]）
    return blank_row


def hamada_colum(ans_candidate,board): #列に着目し浜田ロジックを使う
    flag_hamada_colum =False
    block_candidate_two =hamada_block_two(ans_candidate,board)
    blank_colum =df_blank_colum(ans_candidate,board)
    
    print(block_candidate_two,"2つブロック")
    for i in range(len(block_candidate_two)):
        for s in range(2):#軸の2通り
            shaft_colum =block_candidate_two[i][s][1] #軸の列
            shaft_row_in_block =block_candidate_two[i][s][0]#軸のブロック内の行
            
            shaft_row_1 = blank_colum[shaft_colum][1][0][0]#軸において空白の行
            shaft_row_2 = blank_colum[shaft_colum][1][1][0]#軸において空白の行
            
            
            no_shaft_colum =block_candidate_two[i][1-s][1]#軸でない同じブロック内の空白のものの列
            no_shaft_row =block_candidate_two[i][1-s][0]#軸でない同じブロック内の空白のものの行
            
            if blank_colum[shaft_colum][0] !=2 :#軸の列は空欄が２つ　
                continue
            for colum in range(9):
                print(blank_colum[colum][1],"ffafafafa")
                if blank_colum[colum][0] ==2 and blank_colum[colum][1][0][1] !=shaft_colum:#ある列において空白が２つ　、その列は軸ではない
                    if (len(list(ans_candidate[shaft_row_1][colum]))==2 or len(list(ans_candidate[shaft_row_2][colum]))==2) and len(list(ans_candidate[no_shaft_row][colum]))==2:
                        comfirm_double =ans_candidate[no_shaft_row][no_shaft_colum] | ans_candidate[no_shaft_row][colum] #候補がかぶっているかどうか判断
                        if len(list(comfirm_double))!=3:#候補の種類は3種類でなければならない
                            continue
                        
                        
                        dif_no_shaft_set =comfirm_double-ans_candidate[no_shaft_row][colum]
                        if len(list(dif_no_shaft_set)) ==0:
                            print("error_hamada_colum")
                        dif_no_shaft =dif_no_shaft_set.pop()
                        flag_hamada_colum =True
                        board[no_shaft_row][no_shaft_colum] =dif_no_shaft #ブロック内の軸でない座標を埋める
                        ans_candidate[no_shaft_row][no_shaft_colum] =set([])
                        same_set =ans_candidate[shaft_row_in_block][shaft_colum]-dif_no_shaft_set
                        same =same_set.pop()
                        board[shaft_row_in_block][shaft_colum] =same #ブロック内の軸の座標を埋める
                        ans_candidate[shaft_row_in_block][shaft_colum] =set([])
                        colum_common_sense(ans_candidate,board) #軸のブロック外の座標を埋める
                        leftovers(ans_candidate,board)
    print(flag_hamada_colum)
    return flag_hamada_colum







def hamada_row(ans_candidate,board): #行に着目し浜田ロジックを使う
    flag_hamada_row =False
    block_candidate_two =hamada_block_two(ans_candidate,board)
    blank_colum =df_blank_colum(ans_candidate,board)
    print(block_candidate_two)
    for i in range(len(block_candidate_two)):
        for s in range(2):#軸の2通り
            shaft_colum =block_candidate_two[i][s][1] #軸の列
            if blank_colum[shaft_colum][0] !=2 :#軸の列は空欄が２つ　
                continue
            shaft_row_in_block =block_candidate_two[i][s][0]#軸のブロック内の行
            for row_search in range(2):#軸のブロック内の行を求める
                shaft_row_out_block =blank_colum[shaft_colum][1][row_search][0]#軸のブロック内の行
                if shaft_row_out_block !=shaft_row_in_block:
                    break
            
           # shaft_row_1 = blank_colum[shaft_colum][1][0][0]#軸において空白の行
            #shaft_row_2 = blank_colum[shaft_colum][1][1][0]#軸において空白の行
            
            
            no_shaft_colum =block_candidate_two[i][1-s][1]#軸でない同じブロック内の空白のものの列
            no_shaft_row =block_candidate_two[i][1-s][0]#軸でない同じブロック内の空白のものの行
            
           
            for colum in range(9):
                if colum ==no_shaft_colum:
                    continue
                if blank_colum[colum][0] ==2 and blank_colum[colum][1][0][1] !=shaft_colum:#ある列において空白が２つ　、その列は軸ではない
                    if len(list(ans_candidate[shaft_row_out_block][colum]))==2  and len(list(ans_candidate[no_shaft_row][colum]))==2:
                        comfirm_double =ans_candidate[no_shaft_row][no_shaft_colum] | ans_candidate[no_shaft_row][colum] #候補がかぶっているかどうか判断
                        if len(list(comfirm_double))>=4:
                            continue
                        
                        dif_no_shaft_set =comfirm_double-ans_candidate[no_shaft_row][colum]
                        if len(list(dif_no_shaft_set)) ==0:
                            print("error_hamada_colum")
                        dif_no_shaft =dif_no_shaft_set.pop()
                        flag_hamada_colum =True
                        board[no_shaft_row][no_shaft_colum] =dif_no_shaft #ブロック内の軸でない座標を埋める
                        ans_candidate[no_shaft_row][no_shaft_colum] =set([])
                        same_set =ans_candidate[shaft_row_in_block][shaft_colum]-dif_no_shaft_set
                        same =same_set.pop()
                        board[shaft_row_in_block][shaft_colum] =same #ブロック内の軸の座標を埋める
                        ans_candidate[shaft_row_in_block][shaft_colum] =set([])
                        colum_common_sense(ans_candidate,board) #軸のブロック外の座標を埋める
                        leftovers(ans_candidate,board)
    return flag_hamada_row
                        


                
ans_candidate =ans_candidate_set(board)


if debug_board(board,input_data[number]) is True:
        print("正常です。")
else:
        print(input_data[number],"以上の入力データにエラー")
            

                
                
                
                     
                
n =int(input())
data_input(n)
board =np.array(list(board_all)).reshape(9,9)
ans_candidate =ans_candidate_set(board)          
          

print(ans_candidate)
print(hamada_colum(ans_candidate,board))


hamada_row(ans_candidate,board)   
                


          
print(board)

print()
#412875369698324070375916842187639254200000907000740608700090420000000796020467083